In [ ]:
import os
import openpyxl
import pandas as pd
import yfinance as yf
from glob import glob
from datetime import timedelta, datetime

In [ ]:
df = pd.DataFrame()
require_cols = ['Symbol', 'Trade Date','Trade Type','Quantity','Price','Order ID','Order Execution Time']

In [ ]:
for f in glob("zerodha_exports/tradebook-*.xlsx"):
    data = pd.read_excel(f, skiprows = 14, usecols = require_cols)
    df = df.append(data,ignore_index=True)
df.drop_duplicates('Order ID', inplace=True)
df = df.drop(df[df['Trade Type'] == 'sell'].index)
df.reset_index(inplace=True)
holdings_file = glob(("zerodha_exports/holdings-*.xlsx"))
wb = openpyxl.load_workbook(holdings_file[0])
sh = wb.active
invested_val = sh['C15'].value
present_val = sh['C16'].value
unrealised_percentage = sh['C18'].value

In [ ]:
def find_indexPrice(row):
    tradeDate=str(row['Trade Date'])
    ticker=row['Symbol'] +'.NS'
    print(ticker)
    format = '%Y-%m-%d'
    start = datetime.strptime(tradeDate, format)
    end = start + timedelta(days=1)
    index_Price= yf.download('NIFTYBEES.NS',start,end)['Adj Close'].astype(int)
    print(index_Price[0])
    return index_Price[0]

In [ ]:
index_ClosePrices,stock_LTP_Lst = [],[]
for numb,row in df.iterrows():
    index_Price = find_indexPrice(row)
    index_ClosePrices.append(index_Price)
df["index_ClosePrices"] = index_ClosePrices 

In [ ]:
LTP_index = yf.Ticker('NIFTYBEES.NS').history(period='1d')
index_LTP = LTP_index['Close'][0]

investVal_Lst,indexPnL_Lst = [],[]
def find_PnL(row):
    invest_Val = row['Price']*row['Quantity']
    indexCurrent_Val = index_LTP*(invest_Val/row['index_ClosePrices'])
    index_PnL = indexCurrent_Val - invest_Val
    return invest_Val, index_PnL

In [ ]:
for x,row in df.iterrows():
    invest_Val, index_PnL = find_PnL(row)
    investVal_Lst.append(invest_Val)
    indexPnL_Lst.append(index_PnL)
df["invest_Value"] = investVal_Lst 
df["index_P&L"] = indexPnL_Lst
print(df)
df.to_excel("output.xlsx")

In [ ]:
def pnl_Comparison():
    index_PnL = df['index_P&L'].sum()
    index_PnL_Percentage = (index_PnL/invested_val)*100
    print("Invest Value", invested_val)
    print("Stock P&L",present_val)
    print("Index P&L",index_PnL)
    print("Stock P&L Percentage",unrealised_percentage)
    print("Index P&L Percentage",index_PnL_Percentage)

In [ ]:
pnl_Comparison()